# Examen deel 1 : Data load : data science Syntra, Tim Hellemans, J. Van Erum, juni 2024

# A : nodige modules

In [2]:
# standaard in python
import os 
import datetime
import sys
import json
import random
import calendar
import gzip

In [3]:
# te installeren packages en modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

from scipy import stats
# from scipy.stats import weibull_min
# from scipy.stats import skewnorm
# from scipy.stats import pareto
# from scipy.stats import beta
from scipy.stats import norm

# from sklearn.linear_model import LinearRegression

from tqdm import tqdm

In [4]:
# eigen packages modules
current_working_directory = os.getcwd()                                    
parent_working_directory = os.path.dirname(current_working_directory)      
scripts_directory = os.path.join(parent_working_directory, 'scripts')      
sys.path.append(scripts_directory)                                         
# sys.path.remove(scripts_directory)
# print(sys.path)

import come_in_handy as cih
import my_matplotlib_standards as mypltstd

In [5]:
# eigen standard set up uitvoering
mypltstd.my_matplotlib_pyplot_standards_setup()

# B : functies nog niet in modules

In [6]:
def mijn_functies_maar_niet_in_een_folder():
    returnwaarde = 'Ik doe niks'
    return returnwaarde

# C : opladen van de gegevens

In [7]:
location = '../data/raw/autoproductie/'
filename = 'filename.csv'
file_to_load = os.path.join(location, f"{filename}")
df = pd.read_csv(file_to_load, delimiter=',')


In [8]:
# Breng alle gegevens te samen in een groot dataframe
# df_save = pd.concat(
#          [df_1, df_2],
#           ignore_index=True)
#  save het voor analyse in een Excel sheet
# df_save.to_excel("../data/interim/dd_save.xlsx")

# D : een eerste controle van de gegevens

In [15]:
# df.describe()
# df.describe(include = "all") 
# df.info()
# df.shape
# df.columns
# df.index
# print(len(df))
# print(f"{df['engine'].nunique(dropna=True) = } \n ")    
# print(f"{df['engine'].value_counts() = } \n ")    
# df.head()
# df.types
# df.isnull().sum()
# df.corr()
# df.groupby('kolomnaam').agg({'andere_kolom':'functie'})
# df.groupby('manufacturer').agg({'price' : 'mean'}).sort_values(by='price', ascending=False)


,price
manufacturer,
Dodge,106336.188198
Porsche,86326.070849
Land Rover,53081.193286
Chevrolet,49502.038292
Tesla,49090.937447
Mercedes-Benz,47717.659441
RAM,44046.123373
GMC,40205.348172
Cadillac,39749.609868


# E : de kolom 'productie' valideren en gebruiksklaar maken